# Pinecone Index Demo

#### Creating a Pinecone Index

In [1]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [2]:
import pinecone

/Users/jerryliu/Programming/gpt_index/.venv/lib/python3.10/site-packages/pinecone/index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
api_key = ""
pinecone.init(api_key=api_key, environment="us-west1-gcp")

In [15]:
pinecone.describe_index("quickstart")

IndexDescription(name='quickstart', metric='dotproduct', replicas=1, dimension=1536.0, shards=1, pods=1, pod_type='p1.x1', status={'ready': True, 'state': 'Ready'}, metadata_config=None, source_collection='')

In [14]:
# dimensions are for text-embedding-ada-002
# NOTE: needs dotproduct for hybrid search
pinecone.create_index("quickstart", dimension=1536, metric="dotproduct", pod_type="p1")

In [6]:
pinecone_index = pinecone.Index("quickstart")

#### Load documents, build the GPTPineconeIndex

In [17]:
from gpt_index import GPTPineconeIndex, SimpleDirectoryReader
from IPython.display import Markdown, display

In [18]:
# load documents
documents = SimpleDirectoryReader('../paul_graham_essay/data').load_data()

In [19]:
# set add_sparse_vector=True to compute sparse vectors during upsert
index = GPTPineconeIndex.from_documents(documents, pinecone_index=pinecone_index, add_sparse_vector=True)

If directly passing in client, cannot automatically reconstruct connetion after save_to_disk/load_from_disk.For automatic reload, store PINECONE_API_KEY in env variable and pass in index_name and environment instead.
INFO:gpt_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:gpt_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 17617 tokens
> [build_index_from_nodes] Total embedding token usage: 17617 tokens


#### Query Index

In [20]:
# set Logging to DEBUG for more detailed outputs
response = index.query("What did the author do growing up?", vector_store_query_mode="hybrid")

INFO:gpt_index.token_counter.token_counter:> [query] Total LLM token usage: 4061 tokens
> [query] Total LLM token usage: 4061 tokens
INFO:gpt_index.token_counter.token_counter:> [query] Total embedding token usage: 8 tokens
> [query] Total embedding token usage: 8 tokens


In [21]:
display(Markdown(f"<b>{response}</b>"))

<b>

The author grew up writing short stories, programming on an IBM 1401, and working on microcomputers. He wrote simple games, a program to predict how high his model rockets would fly, and a word processor. He studied philosophy in college, but switched to AI. He reverse-engineered SHRDLU for his undergraduate thesis and wrote a book about Lisp hacking. He visited the Carnegie Institute and realized he could make art that would last. He took art classes at Harvard and applied to RISD and the Accademia di Belli Arti in Florence, where he arrived at an arrangement whereby the students wouldn't require the faculty to teach anything, and in return the faculty wouldn't require the students to learn anything. He also started painting still lives in his bedroom at night, using leftover scraps of canvas.</b>